In [1]:
import pandas as pd
import numpy as np
from collections import Counter

<img src="IMG_0597.jpg" width="28%">
<img src="market_value.png" width="63.5%">

### "If he were to pursue another transfer now, he would be worth a three-digit million amount." - Lothar Matthäus
##### --> Reasons it by referring to his first few premier league goals and a goal for the national team.

***Transfermarkt market values are community based***

# How well can individual player data predict transfer fees in football? What is the optimal timeframe to look back on?
#### Possible implicit questions: How important are negotiation skills?, How big is the role of subjective scouting?

# Data Preprocessing

## Sources
Combination of individual Player data from https://fbref.com/en/, (2017/2018 - 2024/2025) with transfers infos from https://github.com/ewenme/transfers (1992/1993 - 2022/2023)

### Individual Player Stats (last time)

In [2]:
#get data and group by player name
data = pd.read_csv('main.csv', sep=',')
player_groups = dict(tuple(data.groupby('player_name')))

In [4]:
#show data structure
display(data.head())
print(data.shape)
print(f"Unique seasons ({data['season'].nunique()}): {data['season'].unique()}")
print(f"Unique leagues ({data['league'].nunique()}): {data['league'].unique()}")
print('='*100)
print(f"Total players grouped: {len(player_groups)}.")
print('-'*40)
first_player = list(player_groups.keys())[4]
print(f"Example entry for '{first_player}':")
print(player_groups[first_player].head(2))
print('='*100)

,league,season,team,home_away,player_name,shirt_number,nation,position,age,minutes_played,...,total_shots,shots_on_target,blocks,xG,npxG,shot_creating_actions,goal_creating_actions,carries,prog_carries,season_half_idx
0,Premier_League,17_18,Arsenal,home,Alexandre Lacazette,9.0,fr FRA,"FW,LW,LM",26-075,90.0,...,3.0,2.0,1.0,0.3,0.3,6.0,2.0,19.0,1.0,0.0
1,Premier_League,17_18,Arsenal,home,Danny Welbeck,23.0,eng ENG,"AM,FW",26-258,74.0,...,3.0,1.0,0.0,1.0,1.0,4.0,0.0,18.0,3.0,0.0
2,Premier_League,17_18,Arsenal,home,Theo Walcott,14.0,eng ENG,RW,28-148,16.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0
3,Premier_League,17_18,Arsenal,home,Mesut Özil,11.0,de GER,"AM,RM",28-300,90.0,...,3.0,0.0,1.0,0.1,0.1,10.0,0.0,69.0,3.0,0.0
4,Premier_League,17_18,Arsenal,home,Granit Xhaka,29.0,ch SUI,"CM,DM",24-318,90.0,...,4.0,1.0,1.0,0.1,0.1,12.0,2.0,80.0,4.0,0.0


(423662, 108)
Unique seasons (8): ['17_18' '18_19' '19_20' '20_21' '21_22' '22_23' '23_24' '24_25']
Unique leagues (5): ['Premier_League' 'La_Liga' 'Ligue_1' 'Bundesliga' 'Serie_A']
Total players grouped: 7135.
----------------------------------------
Example entry for 'Aaron Hunt':
            league season          team home_away player_name  shirt_number  \
259886  Bundesliga  17_18  Hamburger SV      home  Aaron Hunt          14.0   
260036  Bundesliga  17_18  Hamburger SV      away  Aaron Hunt          14.0   

        nation position     age  minutes_played  ...  total_shots  \
259886  de GER    AM,LW  30-349            76.0  ...          2.0   
260036  de GER       AM  30-355            28.0  ...          0.0   

        shots_on_target  blocks   xG  npxG  shot_creating_actions  \
259886              0.0     1.0  0.1   0.1                    2.0   
260036              0.0     0.0  0.0   0.0                    1.0   

        goal_creating_actions  carries  prog_carries  season_h

### Transfer Infos
Get info on fee, transfer period, player name, involved clubs, loan

Aggrgeate data for the timeframe of 2017 - 2022 

In [5]:
#read individual league transfer data and create transfer_main.csv
files = ['1-bundesliga.csv', 'championship.csv', 'eredivisie.csv', 'liga-nos.csv', 'ligue-1.csv', 'premier-league.csv', 'primera-division.csv', 'serie-a.csv']

main_df = pd.DataFrame()
for i, df in enumerate([pd.read_csv(file) for file in files]):
    print('-'*40)
    print(f'Shape of original dataframe for {files[i]}: {df.shape}')
    df = df.dropna(subset=['fee_cleaned']).copy()

    mask_loan = df['fee'].str.lower().str.contains('loan')
    df['is_loan'] = np.where(mask_loan, 1, 0)

    mask_in = df['transfer_movement'] == 'in'
    df['selling_club'] = np.where(mask_in, df['club_involved_name'], df['club_name'])
    df['buying_club']  = np.where(mask_in, df['club_name'], df['club_involved_name'])

    df['transfer_period'] = df['transfer_period'] + ' ' + df['season']
    df = df[df['year'] >= 2017]

    df = df.drop(columns=['club_name', 'club_involved_name', 'transfer_movement', 'season', 'position', 'age', 'league_name'])
    main_df = pd.concat([main_df, df], ignore_index=True)
    print(f'Shape of cleaned dataframe (Transfers in 2017 or later) for {files[i]}: {df.shape}')

main_df.to_csv('transfers_main.csv', index=False)
print('-'*40)
print(f'Final shape of combined dataframe, data saved to transfers_main.csv: {main_df.shape}')
print(f'Columns of \'transfer_main.csv\': {main_df.columns.tolist()}')
print('-'*40)

----------------------------------------
Shape of original dataframe for 1-bundesliga.csv: (14158, 13)
Shape of cleaned dataframe (Transfers in 2017 or later) for 1-bundesliga.csv: (1686, 9)
----------------------------------------
Shape of original dataframe for championship.csv: (24241, 13)
Shape of cleaned dataframe (Transfers in 2017 or later) for championship.csv: (1819, 9)
----------------------------------------
Shape of original dataframe for eredivisie.csv: (15264, 13)
Shape of cleaned dataframe (Transfers in 2017 or later) for eredivisie.csv: (1524, 9)
----------------------------------------
Shape of original dataframe for liga-nos.csv: (21734, 13)
Shape of cleaned dataframe (Transfers in 2017 or later) for liga-nos.csv: (1859, 9)
----------------------------------------
Shape of original dataframe for ligue-1.csv: (17580, 13)
Shape of cleaned dataframe (Transfers in 2017 or later) for ligue-1.csv: (1777, 9)
----------------------------------------
Shape of original datafram

When scraping we did not capture the matchday of the games.

Since we now need to know when a game was to use the most recent data to predict a fee, a column holding an index indicating a certain season half is needed.

The beginning and end of every game, season and league in main.csv is captured as a tuple of indices.

In [6]:
#get indices for games, seasons, leagues
game_indices = []
current_home = data.loc[0, 'team']
start_idx = 0
for i in range(1, len(data)):
    if data.loc[i, 'home_away'] == 'home' and data.loc[i, 'team'] != data.loc[i-1, 'team'] :
      game_indices.append((start_idx, i - 1))
      start_idx = i
game_indices.append((start_idx, len(data) - 1))

season_indices = []
current_season = data.loc[0, 'season']
start_idx = 0

for i in range(1, len(data)):
    if data.loc[i, 'season'] != data.loc[i-1, 'season'] :
        season_indices.append((start_idx, i))
        start_idx = i + 1
season_indices.append((start_idx, len(data) - 1))

league_indices = []
current_league = data.loc[0, 'league']
start_idx = 0

for i in range(1, len(data)):
    if data.loc[i, 'league'] != data.loc[i-1, 'league'] :
        league_indices.append((start_idx, i))
        start_idx = i + 1
league_indices.append((start_idx, len(data) - 1))
print(f'{len(game_indices)} games: {game_indices}')
print(f'{len(season_indices)} seasons: {season_indices}')
print(f'{len(league_indices)} leagues: {league_indices}')

14245 games: [(0, 27), (28, 55), (56, 82), (83, 109), (110, 136), (137, 164), (165, 192), (193, 220), (221, 248), (249, 276), (277, 304), (305, 332), (333, 358), (359, 386), (387, 412), (413, 440), (441, 468), (469, 496), (497, 523), (524, 551), (552, 578), (579, 606), (607, 634), (635, 662), (663, 690), (691, 718), (719, 745), (746, 773), (774, 801), (802, 829), (830, 857), (858, 885), (886, 913), (914, 941), (942, 968), (969, 996), (997, 1023), (1024, 1050), (1051, 1077), (1078, 1105), (1106, 1132), (1133, 1159), (1160, 1186), (1187, 1213), (1214, 1241), (1242, 1269), (1270, 1297), (1298, 1324), (1325, 1352), (1353, 1380), (1381, 1408), (1409, 1436), (1437, 1464), (1465, 1491), (1492, 1518), (1519, 1545), (1546, 1573), (1574, 1601), (1602, 1627), (1628, 1655), (1656, 1683), (1684, 1711), (1712, 1739), (1740, 1766), (1767, 1794), (1795, 1822), (1823, 1850), (1851, 1878), (1879, 1904), (1905, 1960), (1961, 1988), (1989, 2016), (2017, 2044), (2045, 2072), (2073, 2100), (2101, 2128), (21

After counting the number of games in each season, the first half gets an index and the second the next higher one.

8 seasons --> index ranges from 0 to 15

In [7]:
#create column 'season_half_idx' in main dataframe
season_half_idx = []

for l_start, l_end in league_indices:
    counter = 0
    for s_start, s_end in season_indices:
        if s_start >= l_start and s_end <= l_end:
            length = s_end - s_start + 1
            if length % 2 == 0:
                first_half_length = length // 2
                second_half_length = length - first_half_length
            else:
                first_half_length = length // 2 + 1
                second_half_length = length - first_half_length
            season_half_idx.extend([float(counter)] * first_half_length)
            season_half_idx.extend([float(counter + 1)] * second_half_length)
            counter += 2

if len(season_half_idx) < len(data):
    season_half_idx.extend([counter - 1] * (len(data) - len(season_half_idx)))

data['season_half_idx'] = pd.Series(season_half_idx[1:]).reset_index(drop=True)
display(data.head())
#data.to_csv('main.csv', sep=',', index=False)

,league,season,team,home_away,player_name,shirt_number,nation,position,age,minutes_played,...,total_shots,shots_on_target,blocks,xG,npxG,shot_creating_actions,goal_creating_actions,carries,prog_carries,season_half_idx
0,Premier_League,17_18,Arsenal,home,Alexandre Lacazette,9.0,fr FRA,"FW,LW,LM",26-075,90.0,...,3.0,2.0,1.0,0.3,0.3,6.0,2.0,19.0,1.0,0.0
1,Premier_League,17_18,Arsenal,home,Danny Welbeck,23.0,eng ENG,"AM,FW",26-258,74.0,...,3.0,1.0,0.0,1.0,1.0,4.0,0.0,18.0,3.0,0.0
2,Premier_League,17_18,Arsenal,home,Theo Walcott,14.0,eng ENG,RW,28-148,16.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0
3,Premier_League,17_18,Arsenal,home,Mesut Özil,11.0,de GER,"AM,RM",28-300,90.0,...,3.0,0.0,1.0,0.1,0.1,10.0,0.0,69.0,3.0,0.0
4,Premier_League,17_18,Arsenal,home,Granit Xhaka,29.0,ch SUI,"CM,DM",24-318,90.0,...,4.0,1.0,1.0,0.1,0.1,12.0,2.0,80.0,4.0,0.0


From Summer 2017/2018 to Winter 2022/2023 every transfer period is assigned an index as well.

In [8]:
#match transfer to season half index, create column 'transfer_window_idx' in transfer dataframe
transfer_data = pd.read_csv('transfers_main.csv', sep=',')
window_mask = transfer_data['transfer_period'].str.contains('Winter')
transfer_data['transfer_window_idx'] = 0
for i in np.arange(2017, 2023):
    string = f'{i}/{i+1}'
    season_mask = transfer_data['transfer_period'].str.contains(string)
    transfer_data['transfer_window_idx'] += (i - 2017) * 2 * season_mask 
transfer_data['transfer_window_idx'] += window_mask

transfer_data.to_csv('transfers_main.csv', index=False)

summary = (transfer_data[['transfer_period', 'transfer_window_idx']].drop_duplicates().sort_values(by='transfer_window_idx').reset_index(drop=True))
display(summary)

,transfer_period,transfer_window_idx
0,Summer 2017/2018,0
1,Winter 2017/2018,1
2,Summer 2018/2019,2
3,Winter 2018/2019,3
4,Summer 2019/2020,4
5,Winter 2019/2020,5
6,Summer 2020/2021,6
7,Winter 2020/2021,7
8,Summer 2021/2022,8
9,Winter 2021/2022,9


With the transfer period and season half indices we can easily filter which data to use for a prediction of transfers from certain transfer periods. 

To account for the steadily rising infaltion in football transfer fees, we adjust the fees to the 2022 level.

We mimic a steadily rising level of transfer fees in football by linearly interpolating between the first and last year of our dataset.

In [9]:
#add column with adjusted fees, based on mean fee for each year
transfer_data = pd.read_csv('transfers_main.csv', sep=',')

df = transfer_data[(transfer_data['fee_cleaned'] != 0) & (transfer_data['is_loan'] == 0)]

q = 0.05
df = df[(df['fee_cleaned'] > df['fee_cleaned'].quantile(q)) & (df['fee_cleaned'] < df['fee_cleaned'].quantile(1 - q))] #exclude outliers
base_fee_2022 = df.loc[df['year'] == 2022, 'fee_cleaned'].mean()

print('Inflation factors based on mean transfer fees:')
print(f'2022 base fee: {round(base_fee_2022, 2)}')
inflation_dict = {}
for year in df['year'].unique():
    mean_fee = df.loc[df['year'] == year, 'fee_cleaned'].mean()
    inflation_dict[year] = base_fee_2022 / mean_fee
    if year == 2017 or year == 2022:
        print(year, round(inflation_dict[year], 2), f'(mean fee: {round(mean_fee, 2)})')

print('-'*40)
print('Linear interpolation of inflation factors:')
values = list(inflation_dict.values())
for key, value in zip(sorted(inflation_dict.keys()), np.linspace(values[0], values[-1], len(values))):
    inflation_dict[key] = value
    print(key, round(value, 2))

transfer_data['adjusted_fee'] = (transfer_data['fee_cleaned'] * transfer_data['year'].map(inflation_dict)) #map replaces every year with the corresponding inflation factor
print('-'*40)
print(transfer_data['adjusted_fee'].describe())
display(transfer_data[['player_name', 'fee_cleaned','year', 'adjusted_fee']].head())

Inflation factors based on mean transfer fees:
2022 base fee: 6.53
2017 1.11 (mean fee: 5.86)
2022 1.0 (mean fee: 6.53)
----------------------------------------
Linear interpolation of inflation factors:
2017 1.11
2018 1.09
2019 1.07
2020 1.05
2021 1.02
2022 1.0
----------------------------------------
count    14928.000000
mean         4.381434
std         11.085230
min          0.000000
25%          0.000000
50%          0.334007
75%          3.579349
max        247.164941
Name: adjusted_fee, dtype: float64


,player_name,fee_cleaned,year,adjusted_fee
0,Corentin Tolisso,41.5,2017,46.204257
1,Kingsley Coman,21.0,2017,23.380467
2,Niklas Süle,20.0,2017,22.267112
3,James Rodríguez,13.0,2017,14.473623
4,Serge Gnabry,8.0,2017,8.906845


Now onto the actual contruction of the dataframe used for ML:

1. loop through every transfer in transfer_main.csv

2. get transfer window index and compute relevant season halves from it (we did 1 - 4 relevant season halves)
- transfer in window 'Winter 2019/2020' and 2 season halves are considered relevant --> get data from second season half of 2018/2019 and first season half of 2019/2020

    which translates to:

- transfer in window 5 and x = 2 -> relevant_season_halves = [3, 4]

3. slice out data from player_groups by filtering by transfer_window_idx

4. aggregate data appropiately (percentages --> take mean, else --> take sum)

5. get specific columns (main_position, age_at_transfer)

6. standardize data for numerical stability, check rank of design matrix and drop linearly dependent columns

7. create csv files for ML:

- 4 full dataframes with all transfers (differ in relevant season halves)
- for each number of relevant season halves and each position in ['forward', 'winger', 'midfielder', 'defender', 'goalkeeper'] one dataframe with all columns, one with position specific columns (goals matter more for the fee of a striker or winger than a defender)

In [10]:
#for a certain transfer window index aggregate data from x previous season halves

def aggregate_data(season_halves, player_data):
    slices = pd.DataFrame()
    slices_list = []
    for s in season_halves:
        df_slice = player_data[player_data['season_half_idx'] == s]
        if not df_slice.empty:
            slices_list.append(df_slice)
    if not slices_list:
        return pd.DataFrame()
    slices = pd.concat(slices_list, ignore_index=True)

    agg_dict = {}
    for column in slices.columns:
        if column.endswith('perc'):
            #weights = slice['minutes_played'] / slice['minutes_played'].sum()
            #values = (slice[column] * weights).sum()
            values = slices[column].mean()
            values = round(values, 2)
        else:
            try:
                numeric_vals = pd.to_numeric(slices[column])
                values = round(numeric_vals.sum(), 2)
                values = round(values, 2)
            except:
                values = slices[column].unique().tolist()
                values = ', '.join(map(str, slices[column].dropna().unique().tolist()))
        agg_dict[column] = values
    aggregated_data = pd.DataFrame([agg_dict])
    aggregated_data.drop(columns=['shirt_number', 'season_half_idx', 'player_name'], inplace=True, errors='ignore')
    return aggregated_data

def get_main_position(position_str):
    abbrevs = [p.strip() for p in position_str.split(',') if p.strip()]
    mapped = [position_map.get(p) for p in abbrevs if position_map.get(p)]
    if not mapped:
        return None
    return Counter(mapped).most_common(1)[0][0]

def standardize(X):
    columns = X.columns if isinstance(X, pd.DataFrame) else None
    index = X.index if isinstance(X, pd.DataFrame) else None
    X_np = np.array(X, dtype=float)
    X_np = np.nan_to_num(X_np, nan=0.0, posinf=0.0, neginf=0.0)
    means = np.nanmean(X_np, axis=0)
    stds = np.nanstd(X_np, axis=0)
    stds[stds == 0] = 1
    X_z = (X_np - means) / stds
    X_z = np.clip(X_z, -1e6, 1e6)
    if columns is not None:
        X_z = pd.DataFrame(X_z, columns=columns, index=index)
    return X_z

def reduce_to_full_rank(X_z, ml_df):
    rank = np.linalg.matrix_rank(X_z)
    print(f'Rank of design matrix: {rank}, Number of columns: {X_z.shape}')
    if rank != X_z.shape[1]:
        Q, R = np.linalg.qr(X_z.to_numpy())
        independent_idx = np.abs(R.diagonal()) > 1e-10
        independent_cols = X_z.columns[independent_idx]
        X_z_reduced = X_z[independent_cols]
        new_rank = np.linalg.matrix_rank(X_z_reduced)
        if new_rank == X_z_reduced.shape[1]:
            print(f'Reduced design matrix with shape {X_z_reduced.shape} has full rank: {new_rank}.')
            print(f'{len(set(X_z.columns) - set(independent_cols))} columns removed: {set(X_z.columns) - set(independent_cols)}')
            ml_df = pd.concat([ml_df[['is_loan', 'age_at_transfer', 'main_position', 'selling_club', 'buying_club']], X_z_reduced], axis=1)
        else:
            print(f'Reduction did not achieve full rank. Rank: {new_rank}, Shape: {X_z_reduced.shape}')
    else:
        print(f'Design matrix has full rank, no reduction necessary: {rank}.')
    return ml_df

position_map = {
    'FW': 'forward',
    'ST': 'forward',
    'CF': 'forward',
    'LW': 'winger',
    'RW': 'winger',
    'LM': 'winger',
    'RM': 'winger',
    'RB': 'defender',
    'LB': 'defender',
    'RWB': 'defender',
    'LWB': 'defender',
    'CB': 'defender',
    'CM': 'midfielder',
    'DM': 'midfielder',
    'AM': 'midfielder',
    'GK': 'goalkeeper'
}

positional_columns = {
    'forward': [
        'goals', 'xG', 'npxG', 'shots_on_target', 'total_shots',
        'penalty_kicks_scored', 'penalty_kicks_attempted',
        'xA', 'key_passes', 'assists',
        'goal_creating_actions', 'shot_creating_actions',
        'prog_passes_received', 'total_passes_received',
        'take_ons_attempted', 'take_ons_successful', 'take_ons_success_perc',
        'touches_att_pen_area', 'touches_att_3rd',
        'carries_into_penalty_area', 'prog_carries',
        'offsides', 'fouls_drawn', 'shot_creation_ratio'
    ],
    'winger': [
        'goals', 'xG', 'shots_on_target', 'xA', 'assists', 'key_passes',
        'crosses', 'crosses_into_penalty_area', 'passes_final_third',
        'pass_into_penalty_area', 'prog_passes', 'shot_creating_actions',
        'goal_creating_actions', 'take_ons_attempted', 'take_ons_successful',
        'take_ons_success_perc', 'carries_into_final_3rd', 'carries_into_penalty_area',
        'prog_carries', 'touches_att_3rd', 'touches_att_pen_area',
        'total_miscontrols', 'dispossessed', 'offsides', 'shot_creation_ratio'
    ],
    'midfielder': [
        'assists', 'xA', 'xAG', 'key_passes', 'passes_attempted', 'passes_completed',
        'pass_compl_perc', 'prog_passes', 'passes_final_third',
        'through_balls', 'switches', 'prog_carries',
        'total_carries', 'total_carrying_distance', 'prog_carrying_distance',
        'carries_into_final_3rd', 'tackles', 'interceptions', 'ball_recoveries',
        'blocks', 'tackles_and_interceptions', 'fouls_committed', 'fouls_drawn',
        'touches_mid_3rd', 'touches_att_3rd', 'touches_def_3rd',
        'shot_creating_actions', 'goal_creating_actions', 'shot_creation_ratio'
    ],
    'defender': [
        'tackles', 'tackles_def_3rd', 'tackles_and_interceptions', 'interceptions',
        'clearances', 'blocks', 'shots_blocked', 'passes_blocked',
        'aerials_won', 'aerials_lost', 'aerials_won_perc', 'errors', 'own_goals',
        'fouls_committed', 'yellow_cards', 'red_cards', 'second_yellow_card',
        'passes_attempted', 'passes_completed', 'pass_compl_perc',
        'short_pass_compl_perc', 'medium_pass_compl_perc', 'long_pass_compl_perc',
        'total_pass_distance', 'total_prog_pass_distance', 'prog_passes',
        'crosses', 'prog_carries', 'carries_into_final_3rd',
        'touches_def_pen_area', 'touches_def_3rd', 'ball_recoveries'
    ],
    'goalkeeper': [
        'touches_def_pen_area', 'touches_def_3rd',
        'passes_attempted', 'passes_completed', 'pass_compl_perc',
        'long_pass_attempted', 'long_pass_completed', 'long_pass_compl_perc',
        'clearances', 'errors', 'own_goals'
    ]
}

base_columns = ['is_loan', 'age_at_transfer', 'main_position', 'selling_club', 'buying_club', 'adjusted_fee', 'minutes_played']

mode = ''
for x in range(1, 5, 1):
    print('-'*100)
    print(f'{x} relevant season halves:')

    #aggregate relevant data fro eac transfer and create ml dataframe
    ml_df = pd.DataFrame()
    for i, single_transfer_data in transfer_data.iterrows():
            
        single_transfer_data = single_transfer_data.to_frame().T
        player_name = single_transfer_data['player_name'].item()  

        if player_name in player_groups.keys():
            player_data = player_groups[player_name]

        #find transfer window idx for player
        #relevant_season_halves = get_season_halves(x, single_transfer_data)
        transfer_window_idx = single_transfer_data['transfer_window_idx'].values[0]
        relevant_season_halves = [i for i in list(range(transfer_window_idx - x, transfer_window_idx)) if i >= 0] #no negative indices
        #get aggregated data for relevant season halves
        agg_data = aggregate_data(relevant_season_halves, player_data)

        if not agg_data.empty:
            agg_data = pd.concat([single_transfer_data.reset_index(drop=True), agg_data.reset_index(drop=True)], axis=1) # pd.concat([single_transfer_data, agg_data], axis=1) #put together single transfer data
            ml_df = pd.concat([ml_df, agg_data], ignore_index=True) #add single transfer data to other transfers
    
    #create new relevant columns and drop irrelevant ones
    ml_df['main_position'] = ml_df['position'].apply(get_main_position)
    ml_df['shot_creation_ratio'] = round(ml_df['shot_creating_actions'] / (ml_df['carries_into_final_3rd'] + ml_df['passes_final_third'] + ml_df['touches_att_3rd']).replace(0, np.nan), 4)
    ml_df['age_at_transfer'] = ml_df['age'].astype(str).str[-6:-4]
    ml_df.drop(['player_name', 'transfer_period', 'fee_cleaned', 'country', 'fee', 'transfer_window_idx',
                'age', 'position', 'year', 'league', 'season', 'team', 'home_away', 'nation'], axis=1, inplace=True)
    
    #order base columns
    for i, col in enumerate(base_columns):
        ml_df.insert(i, col, ml_df.pop(col))

    # check rank of ml_df
    X = ml_df.iloc[:, 5:].apply(pd.to_numeric, errors='coerce')
    X_z = standardize(X)
    ml_df = reduce_to_full_rank(X_z, ml_df)

    if mode == 'csv':
        ml_df.to_csv(f'transfer_ml_data_last_{x}_season_halves.csv', index=False)
        print(f'Csv file \'transfer_ml_data_last_{x}_season_halves.csv\' with shape {ml_df.shape} created.')

    #create position based csv files
    positions = ['forward', 'winger', 'midfielder', 'defender', 'goalkeeper']
    for position in positions:
        subset = ml_df[ml_df['main_position'] == position]
        if mode == 'csv':
            subset.to_csv(f'transfer_ml_data_last_{x}_season_halves_{position}_all_columns.csv', index=False)
            print(f'Csv file \'transfer_ml_data_last_{x}_season_halves_{position}_all_columns.csv\' with shape {subset.shape} created.')
        
        #only include certain columns
        cols_to_keep = [c for c in base_columns + positional_columns[position] if c in subset.columns]
        subset = subset[cols_to_keep]
        if mode == 'csv':
            subset.to_csv(f'transfer_ml_data_last_{x}_season_halves_{position}_relevant_columns.csv', index=False)
            print(f'Csv file \'transfer_ml_data_last_{x}_season_halves_{position}_relevant_columns.csv\' with shape {subset.shape} created.')
print('-'*100)
display(ml_df.head())

----------------------------------------------------------------------------------------------------
1 relevant season halves:
Rank of design matrix: 92, Number of columns: (6004, 100)
Reduced design matrix with shape (6004, 92) has full rank: 92.
8 columns removed: {'prog_carries', 'shots_blocked', 'challenges_lost', 'passes_offside', 'tackles_att_3rd', 'blocks', 'carries', 'tackles_and_interceptions'}
----------------------------------------------------------------------------------------------------
2 relevant season halves:
Rank of design matrix: 92, Number of columns: (6721, 100)
Reduced design matrix with shape (6721, 92) has full rank: 92.
8 columns removed: {'prog_carries', 'shots_blocked', 'challenges_lost', 'passes_offside', 'tackles_att_3rd', 'blocks', 'carries', 'tackles_and_interceptions'}
----------------------------------------------------------------------------------------------------
3 relevant season halves:
Rank of design matrix: 92, Number of columns: (7455, 100)
R

,is_loan,age_at_transfer,main_position,selling_club,buying_club,adjusted_fee,minutes_played,yellow_cards,red_cards,second_yellow_card,...,goals,penalty_kicks_scored,penalty_kicks_attempted,total_shots,shots_on_target,xG,npxG,shot_creating_actions,goal_creating_actions,shot_creation_ratio
0,0,29,forward,TSG Hoffenheim,Bayern Munich,0.748780,-0.691288,-0.249131,-0.428910,-0.297688,...,0.130868,0.530172,0.390944,-0.073016,0.110951,0.010004,-0.086585,-0.242836,-0.292427,0.473341
1,1,20,winger,Everton,RB Leipzig,-0.395452,-1.125065,-0.938979,-0.428910,-0.297688,...,-0.559465,-0.264880,-0.293429,-0.684357,-0.667849,-0.581118,-0.597798,-0.814801,-0.588945,-0.487190
2,0,32,defender,RB Leipzig,Celtic,-0.336867,-1.167161,-0.938979,-0.428910,-0.297688,...,-0.559465,-0.264880,-0.293429,-0.767721,-0.667849,-0.599591,-0.619099,-0.851702,-0.737204,-0.751166
3,1,24,forward,Chelsea,Borussia Dortmund,-0.303913,-1.075037,-0.938979,-0.428910,-0.297688,...,-0.214298,-0.264880,-0.293429,-0.573204,-0.455449,-0.470283,-0.469995,-0.777900,-0.737204,-0.287931
4,0,28,forward,Borussia Dortmund,Arsenal,5.394362,-0.413695,-0.479081,1.697734,2.690801,...,1.684118,2.120276,2.444063,0.705054,0.960550,2.392968,2.171275,-0.187485,0.745385,-0.053475
